In [1]:
import nltk
import re
import numpy
from nltk.book import *
from nltk.corpus import PlaintextCorpusReader
import os

######################################## DATA IMPORT FUNCTION #########################################

def corpusSize(corpusname):
    corpussize = len(PlaintextCorpusReader(corpusname, '.*').fileids())
    return corpussize

def listFileName(corpusname, corpussize):
    fileName = []
    corpusfiles = PlaintextCorpusReader(corpusname, '.*')
    corpusfileid = corpusfiles.fileids()
    for x in range(corpussize):
        fileID = corpusfileid[x]
        #.strip('.txt')
        fileName.append(fileID)
    return fileName

def listDirectory(corpusname,corpussize,listName=[]):
    fileDirectory = []
    for x in range(corpussize):
        directory = corpusname + '/' + listName[x]
        fileDirectory.append(directory)
    return fileDirectory

# listDirectory = listDirectory('pd',11 ,listName)

def stringCorpus(corpusname,fileDirectory=[]):
    corpusstr = ""
    files = PlaintextCorpusReader(corpusname,'.*')
    corpussize = len(files.fileids())
    for x in range (1,corpussize):
        fileName = open(fileDirectory[x])
        fileName = fileName.read()
        corpusstr = corpusstr + fileName
    return corpusstr

def listCorpus(corpusname,fileDirectory=[]):
    corpuslist = [[]]
    files = PlaintextCorpusReader(corpusname,'.*')
    corpussize = len(files.fileids())
    for x in range (1,corpussize):
        fileName = open(fileDirectory[x])
        fileName = fileName.read()
        ######## TOKENIZE: #######
        corpuslist.append(re.split('\W+',fileName))
    return corpuslist

def toLower(corpussize, corpuslist = []):
    corpuslower = [[]]
    for i in range (1,corpussize):
        file = corpuslist[i]
        file = [w.lower() for w in file]
        corpuslower.append(file)
    return corpuslower


*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.


LookupError: 
**********************************************************************
  Resource u'corpora/gutenberg' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/Users/wandi/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************

In [1]:
###################################### IMPORT PD & P4R CORPUS ########################################

pdischarge='pd'
corpussize = corpusSize(pdischarge)
fileList = listFileName (pdischarge,corpussize)
directoryList = listDirectory(pdischarge, corpussize, fileList)
corpusList = listCorpus(pdischarge, directoryList)
corpusString = stringCorpus(pdischarge, directoryList)
pd = corpusList
pd = toLower(corpussize,pd)

# print (pd[1])

p4rname = 'p4r'
p4rsize = corpusSize(p4rname)
p4rName = listFileName(p4rname,p4rsize)
p4rDirectory = listDirectory(p4rname, p4rsize, p4rName)
p4rCorpus = listCorpus(p4rname, p4rDirectory)
p4r = toLower(p4rsize,p4rCorpus)

# print (p4r[1])



NameError: name 'corpusSize' is not defined

In [2]:
################################## PATIENT DE-INDENTIFICATION FUNCTION ############################################
# two kinds of patient ID extractors were used to achieve the proper index because the corpus of discharge reports
# were written in 2 different types of formats

############## FIRST TYPE OF DISCHARGE REPORT (FORMAT 1)###########
def listOfID1 (beginning, end, corpuslist = [],idList = [[]]):
    for x in range (beginning, end):
        file = corpuslist[x]
        aindex = file.index('patient')
        zindex = file.index("author")
        pID = []
        pID = file[aindex:zindex]
        idList.append(pID)
    return idList

############## SECOND TYPE OF DISCHARGE REPORT (FORMAT 2)###########
def listOfID2 (beginning, end, corpuslist = [],idList = [[]]):
    for x in range (beginning, end):
        file = corpuslist[x]
        aindex = file.index('patient')
        zindex = file.index("diagnoses") #different stop index
        pID = []
        pID = file[aindex:zindex-1]
        idList.append(pID)
    return idList

####### NEW CORPUS WITH PATIENT INFO DE-IDED #######
def deIdentify (corpussize, patientInfo = [], corpuslist = []):
    dIDed = []
    for i in range (corpussize):
        result= list(filter(lambda x: x not in patientInfo[i], corpuslist[i]))
        dIDed.append(result)
    return dIDed

In [3]:
#PD CORPUS DE-ID
pdIDs = [[]]
pdIDs = listOfID1(1,5,pd,pdIDs)
pdIDs = listOfID2(5,9,pd,pdIDs)
pdIDs = listOfID1(9,corpussize,pd,pdIDs)
corpusDeIDed = deIdentify(corpussize, pdIDs, pd)

# for x in range (1,corpussize):
# print (pdIDs[2], "-----------------------")

#P4R CORPUS De-ID

p4ID= pdIDs[4]
p4rDeIDed = [[]]
for i in range (1,p4rsize):
    result = list(filter(lambda x: x not in p4ID, p4r[i]))
    p4rDeIDed.append(result)

# print(len(p4rDeIDed))
print ("Total of ", len(p4ID), " tokens associated with patient id were extracted, sample of patient MR report after de-identification: \n \n",
       p4rDeIDed[1][140:200]) #sample selection of data after de-identification 

NameError: name 'pd' is not defined

In [4]:
################### STEMMING (optional) ###################
from nltk.stem import WordNetLemmatizer

def stemmer(filelength,file = []):
    stemmedfile = []
    wnstem = WordNetLemmatizer()
    for i in range (filelength):
        stem = wnstem.lemmatize(file[i])
        stemmedfile.append(stem)
    return stemmedfile

pdStemmed = [[]]

for x in range (1,corpussize):
    stemmed = stemmer(len(corpusDeIDed[x]),corpusDeIDed[x])
    pdStemmed.append(stemmed)

print (corpusDeIDed[1][30:120],"\n",pdStemmed[1][30:120]) # does not seem to do anything 

NameError: name 'corpussize' is not defined

In [ ]:
################## PLUMBING ##################
import re 

# remove non-semantic tokens
pdwoNum = [s for s in p4rDeIDed[1] if not re.search(r'\d',s)]

# remove plumbing words (see discussion)
plumbing = []
fdist = FreqDist(pdwoNum) #file = corpuslist[x]
for x in range (0,len(pdwoNum)):
    if (fdist[pdwoNum[x]]>=15 and pdwoNum[x] not in plumbing):
        plumbing.append(pdwoNum[x])
print ("Total of ", len(plumbing), " plumbing words extracted. Sample of plumbing words: \n", plumbing[1:10]) #selection sample

In [5]:
################### BI/TRIGRAMS ######################
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(pdwoNum) 
finder.apply_freq_filter(2)
ignored_words = plumbing #filter out plumbing words
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in ignored_words)
print("top 10 pairs of trigrams:")
finder.nbest(trigram_measures.pmi, 20) 
#next step: filter using tags / categories

NameError: name 'nltk' is not defined

In [6]:
################# K MEANS CLUSTER ######################
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint
import string
import collections

pd01=open('pd/p01d.txt')
pd01=pd01.read()

def processFile(file):
    fileTokened = tokenizer.tokenize(file)
    fileLowered = [w.lower() for w in fileTokened]
    fileLength = len(fileLowered)
    stemmedfile = []
    wnstem = WordNetLemmatizer()
    for i in range (fileLength):
        stem = wnstem.lemmatize(file[i])
        stemmedfile.append(stem)
    return stemmedfile

def cluster_texts(texts, clusters=3):
    """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
    vectorizer = TfidfVectorizer(tokenizer=processFile,
                                 stop_words=stopwords.words('english'),
                                 max_df=0.9,
                                 min_df=0,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    km_model = KMeans(n_clusters=clusters)
    km_model.fit(tfidf_model)
    order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(clusters):
        print ("top 10 items in cluster %d:" % i)
        for ind in order_centroids[i,:10]:
            print (" %s"% terms[ind])
    clustering = collections.defaultdict(list)
 
    for idx, label in enumerate(km_model.labels_):
        clustering[label].append(idx)
 
    return clustering
 
###### PRINTING THING ######

if __name__ == "__main__":
    articles = pd01
    n = 3
    clusters = cluster_texts(articles, n)
#     print(dict(clusters))
    for i in range (n):
        print("cluster",i, ":" ,len(clusters[i]) ,"items")

IOError: [Errno 2] No such file or directory: 'pd/p01d.txt'

In [7]:
############################### LDA MODEL ###################################
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
from gensim import corpora, models

# read files
txt1 = open('p4r/p4mr1.txt')

mr1str = txt1.read()

txt2 = open('p4r/p4mr2.txt')

mr2str = txt2.read()

txt3 = open('p4r/p4mr3.txt')

mr3str = txt3.read()

txt4 = open('p4r/p4mr4.txt')

mr4str = txt4.read()

txt5 = open('p4r/p4mr5.txt')

mr5str = txt5.read()

# create sample documents
doc_a = mr1str
doc_b = mr2str
doc_c = mr3str
doc_d = mr4str
doc_e = mr5str

# load tokenizer 
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = stopwords.words('english')

# Create plumbing words list
pw = plumbing

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
        
# loop through document list
texts = []
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # patient de-ID
    deID_tokens = list(filter(lambda x: x not in p4ID, tokens))
    
    # remove plumbing words
    plumbed_tokens = [i for i in deID_tokens if not i in pw]
    
    # remove stop words from tokens
    stopped_tokens = [i for i in plumbed_tokens if not i in en_stop]
    
    # remove numbers (non-semantic) tokens
    denummed_tokens = [s for s in stopped_tokens if not re.search(r'\d',s)]
    
    # add tokens to list
    texts.append(denummed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=1, id2word = dictionary, passes=20)

print(ldamodel.print_topics(num_topics=1, num_words=15))

ImportError: No module named gensim

In [8]:
############################################### MISC. TESTERS #########################################################

############# CONVERT TO LOWER CASE ####################

def toLower(fizesize, file = []):
    fl = []
    fl = [w.lower() for w in file]
    return fl


########### REMOVE PUNCTUATIONS ############
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(file)


######## FILTER OUT STOP WORDS ###############
from nltk.corpus import stopwords
sw = stopwords.words('english')

######## LOCATE PLUMBING WORDS ###########
plumbing = []
fdist = FreqDist(file) #file = corpuslist[x]
for x in range (0,n):
    if (fdist[file[x]]>=5):
        plumbing.append(file[x])
    
######## FILTER OUT PLUMBING/STOP WORDS ########
fileFiltered = filter(lambda x: x not in plumbing, file)

############# STEMMING ########################

from nltk.stem import WordNetLemmatizer

def stemmer(filelength,file = []):
    stemmedfile = []
    wnstem = WordNetLemmatizer()
    for i in range (filelength):
        stem = wnstem.lemmatize(file[i])
        stemmedfile.append(stem)
    return stemmedfile


mrlowered = toLower(len(mrfilter),mrfilter)
mrstemmed = stemmer(len(mrfilter),mrlowered)
print (mrstemmed)

#################### BIGRAMS ##########################
finder = BigramCollocationFinder.from_words(mrkwo)
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
scored = finder.score_ngrams(bigram_measures.raw_freq)
sorted(bigram for bigram, score in scored) 

TypeError: expected string or buffer